In [2]:
!pip3 install catboost

     ---------------------------------------- 74.0/74.0 MB 2.5 MB/s eta 0:00:00


In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
%matplotlib inline

In [6]:
df = pd.read_csv("features_3_sec.csv")

label2index = dict()
index2label = dict()
for i,unq_label in enumerate(df.label.unique()):
    label2index[unq_label] = i
    index2label[i] = unq_label
df.label = [label2index[l] for l in df.label]
df = df.sample(frac=1).reset_index(drop=True)
df.drop(labels = ['filename','length'], axis=1, inplace = True)
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
my_scaler = StandardScaler()
df_X = pd.DataFrame(my_scaler.fit_transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(df_X, y, train_size=0.9, stratify = y)

# Model Training

In [7]:
cat = CatBoostClassifier(loss_function='MultiClass', eval_metric='Accuracy', verbose=False)

cat.fit(X_train,y_train)
y_pred = cat.predict(X_test)
acc_cat = accuracy_score(y_test,y_pred)
acc_cat

0.9059059059059059

In [8]:
# cat_hyp = CatBoostClassifier(loss_function='MultiClass', eval_metric='Accuracy', verbose=False, iterations= 1000, learning_rate = 0.1,depth = 10)
# cat_hyp.fit(X_train,y_train)
# y_pred = cat_hyp.predict(X_test)
# acc_cat_hyp = accuracy_score(y_test,y_pred)
# acc_cat_hyp

In [9]:
rforest_normal = RandomForestClassifier(n_estimators=1000)
rforest_normal.fit(X_train,y_train)
y_pred = rforest_normal.predict(X_test)
acc_rforest = accuracy_score(y_test,y_pred)
acc_rforest

0.8838838838838838

In [10]:
rforest_tuned = RandomForestClassifier(n_estimators=500,max_depth=50,max_features = 'auto',min_samples_leaf = 4,min_samples_split = 10,bootstrap = True)
rforest_tuned.fit(X_train,y_train)
y_pred_rforest = rforest_tuned.predict(X_test)
acc_rforest_tuned = accuracy_score(y_test,y_pred_rforest)
acc_rforest_tuned

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.8528528528528528

In [11]:
ada1 = AdaBoostClassifier(base_estimator = rforest_normal,n_estimators=5)
ada1.fit(X_train,y_train)
y_pred = ada1.predict(X_test)
acc_ada_1 = accuracy_score(y_test,y_pred)
acc_ada_1

0.9039039039039038

In [12]:
ada2 = AdaBoostClassifier(base_estimator = rforest_tuned,n_estimators=5)
ada2.fit(X_train,y_train)
y_pred = ada2.predict(X_test)
acc_ada_2 = accuracy_score(y_test,y_pred)
acc_ada_2

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

0.9049049049049049

In [13]:
sgd = SGDClassifier(max_iter=500)
sgd_final = BaggingClassifier(base_estimator = sgd,
                          n_estimators = 10,
                          max_samples=0.63, oob_score=True)
sgd_final.fit(X_train,y_train)
y_pred_ada = sgd_final.predict(X_test)
acc = accuracy_score(y_test,y_pred_ada)
acc

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:747: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:753: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.6796796796796797

In [14]:
gaussian_NB = GaussianNB()

gaussian_NB_final = BaggingClassifier(base_estimator = gaussian_NB,
                          n_estimators = 10,
                          max_samples=0.63, oob_score=True)
gaussian_NB_final.fit(X_train,y_train)
y_pred_ada = gaussian_NB_final.predict(X_test)
acc = accuracy_score(y_test,y_pred_ada)
acc

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:747: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:753: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.5215215215215215

In [38]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('r_forest_tuned', rforest_tuned),('ada1',ada1),('ada2',ada2),('cat',cat)],
    voting='soft'
)

# Train the voting classifier on the training set
voting_clf.fit(X_train, y_train)

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

VotingClassifier(estimators=[('r_forest_tuned',
                              RandomForestClassifier(max_depth=50,
                                                     max_features='auto',
                                                     min_samples_leaf=4,
                                                     min_samples_split=10,
                                                     n_estimators=500)),
                             ('ada1',
                              AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=1000),
                                                 n_estimators=5)),
                             ('ada2',
                              AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=50,
                                                                                       max_features='auto',
                                                                                       min_samples_leaf=4,
                                                                                       min_samples_split=10,
                                                                                       n_estimators=500),
                                                 n_estimators=5)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x0000021978588AC0>)],
                 voting='soft')

In [39]:
voting_clf

VotingClassifier(estimators=[('r_forest_tuned',
                              RandomForestClassifier(max_depth=50,
                                                     max_features='auto',
                                                     min_samples_leaf=4,
                                                     min_samples_split=10,
                                                     n_estimators=500)),
                             ('ada1',
                              AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=1000),
                                                 n_estimators=5)),
                             ('ada2',
                              AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=50,
                                                                                       max_features='auto',
                                                                                       min_samples_leaf=4,
                                                                                       min_samples_split=10,
                                                                                       n_estimators=500),
                                                 n_estimators=5)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x0000021978588AC0>)],
                 voting='soft')

In [40]:
voting_clf.estimators_

[RandomForestClassifier(max_depth=50, max_features='auto', min_samples_leaf=4,
                        min_samples_split=10, n_estimators=500),
 AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=1000),
                    n_estimators=5),
 AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=50,
                                                          max_features='auto',
                                                          min_samples_leaf=4,
                                                          min_samples_split=10,
                                                          n_estimators=500),
                    n_estimators=5),

In [ ]:
#this method gives error of mismatched shape of predicted and expected values 
# final_y_ = voting_clf.predict(X_test)
# acc_voting = accuracy_score(y_test,final_y_)
# acc_voting 

#using this method hepls to solve the problem
final_y_proba = voting_clf.predict_proba(X_test)
final_y_ = np.argmax(final_y_proba, axis=1)
acc_voting = accuracy_score(y_test, final_y_)

In [ ]:
acc_voting

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,classification_report,recall_score,f1_score

In [ ]:
precision_voting = precision_score(y_test, final_y_,average='macro')
recall_voting = recall_score(y_test, final_y_,average='macro')
f1_voting = f1_score(y_test, final_y_,average='macro')

In [ ]:
print(precision_voting)
print(recall_voting)
print(f1_voting)

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
# my_classes = df['label'].unique()
my_classes = ['blues','classical','country','disco',\
              'hiphop','jazz','metal','pop','reggae','rock']

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=my_classes, yticklabels=my_classes, ax=ax)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
plt.show()

In [ ]:
classification_report(y_test,y_pred)
classification_report

In [ ]:
# Cat Boost Metrics

In [60]:
y_pred = cat.predict(X_test)
precision_cat = precision_score(y_test, y_pred,average='macro')
recall_cat = recall_score(y_test, y_pred,average='macro')
f1_cat = f1_score(y_test, y_pred,average='macro')
print(precision_cat)
print(recall_cat)
print(f1_cat)

0.9061464517176162
0.905888888888889
0.9057482304891661


In [ ]:
# Random Forest No tuned Metrics

In [61]:
y_pred = rforest_normal.predict(X_test)
precision_rf_normal = precision_score(y_test, y_pred,average='macro')
recall_rf_normal = recall_score(y_test, y_pred,average='macro')
f1_rf_normal = f1_score(y_test, y_pred,average='macro')
print(precision_rf_normal)
print(recall_rf_normal)
print(f1_rf_normal)

0.883615332798616
0.8838585858585859
0.8825990861020531


In [ ]:
# RF tuned metrics

In [62]:
y_pred = rforest_tuned.predict(X_test)
precision_rf_tuned = precision_score(y_test, y_pred,average='macro')
recall_rf_tuned = recall_score(y_test, y_pred,average='macro')
f1_rf_tuned = f1_score(y_test, y_pred,average='macro')
print(precision_rf_tuned)
print(recall_rf_tuned)
print(f1_rf_tuned)

0.8532061294518283
0.8528282828282828
0.8509411721449979


In [ ]:
# Ada 1 metrics

In [63]:
y_pred = ada1.predict(X_test)
precision_ada1 = precision_score(y_test, y_pred,average='macro')
recall_ada1 = recall_score(y_test, y_pred,average='macro')
f1_rf_ada1 = f1_score(y_test, y_pred,average='macro')
print(precision_ada1)
print(recall_ada1)
print(f1_rf_ada1)

0.9037316886564817
0.903878787878788
0.9026984004107714


In [ ]:
# Ada 2 metrics

In [64]:
y_pred = ada2.predict(X_test)
precision_ada2 = precision_score(y_test, y_pred,average='macro')
recall_ada2 = recall_score(y_test, y_pred,average='macro')
f1_ada2 = f1_score(y_test, y_pred,average='macro')
print(precision_ada2)
print(recall_ada2)
print(f1_ada2)

0.9044757568198539
0.9048787878787878
0.9044703571623511


In [ ]:
#sgd metrics

In [65]:
y_pred = sgd_final.predict(X_test)
precision_sgd = precision_score(y_test, y_pred,average='macro')
recall_sgd = recall_score(y_test, y_pred,average='macro')
f1_sgd = f1_score(y_test, y_pred,average='macro')
print(precision_sgd)
print(recall_sgd)
print(f1_sgd)

0.6804992548903198
0.6796060606060605
0.6777252330146981


In [ ]:
#GaussianNB metrics

In [66]:
y_pred = gaussian_NB_final.predict(X_test)
precision_gaussian_NB = precision_score(y_test, y_pred,average='macro')
recall_gaussian_NB = recall_score(y_test, y_pred,average='macro')
f1_gaussian_NB = f1_score(y_test, y_pred,average='macro')
print(precision_gaussian_NB)
print(recall_gaussian_NB)
print(f1_gaussian_NB)

0.5298774897495699
0.5216161616161616
0.506067853805836


In [42]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(voting_clf, f)

In [43]:
with open('model.pkl', 'rb') as f:
    model1 = pickle.load(f)

MemoryError: could not allocate 356240 bytes

In [44]:
y_pred1 = model1.predict(X_test)
acc1 = accuracy_score(y_test,y_pred1)
print(acc1)

NameError: name 'model1' is not defined